In [12]:
import requests  
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim #we are using GeoPy to get latitude and longitude of any city
        


def search_loc(search_link):
    try:
        
        r1 = requests.get(search_link)
        soup = BeautifulSoup(r1.content, 'html.parser')
        table = soup.find('div', attrs = {'class':'byline'})

        table=str(table)

        test=table.split("<dt>")

        x=test[1]

        for i in range(len(x)):
            if x[i]=='<':
                y=i
                break

        return x[:i]
    
    except (AttributeError , IndexError, TypeError):
        pass

def search_latitude_longitude(city):

    try:
        
        if len(city)>1:


            geolocator = Nominatim(user_agent="cawakiy304@ainbz.com") 


            #If you are making large numbers of request please include a valid email address or 
            #alternatively include your email address as part of the User-Agent string.

            cityname=city+",India"

            location = geolocator.geocode(cityname)
            
            
            return (location.latitude, location.longitude)
        else:
            return 0
        
    except(AttributeError , IndexError ,TypeError):
        pass

def webscrappingfun():
    
    df1=pd.read_csv(r'static/assets/data/webscrappeddata.csv') #file is being read.
    
    
    linkarr=df1['link'].tolist()
    
    flag=0
    
    result=[]

    X_array=[]

    for page in range(2):
        
        search_string="https://www.indiatoday.in/crime?page="+str(page)
        print("\n","--------- Page:",page,":",search_string,"\n")
        r = requests.get(search_string)
        soup = BeautifulSoup(r.content, 'html.parser')
        table = soup.find_all('div', attrs = {'class':'detail'})
        
        for i in table:
            tup=[]
            i=str(i)
            res=i.split('"')
            news_heading=res[5]
            
            
            search_link="https://www.indiatoday.in"+res[7]
            
            tup.append(news_heading)

            if search_link in linkarr:
                print("you already have latest data")
                flag=1
                break
            else:
                tup.append(search_link)

                linkarr.append(search_link)

                print("Link:", search_link)

                #print("\n","  Coordinates:",(location.latitude, location.longitude))


                if "video" in search_link:
                    pass

                else:
                    city=search_loc(search_link)
                print("City Extracted:" ,city)


                if city!=None:
                    tup.append(city)
                    tup.append(search_latitude_longitude(city))
                   # X_array.append(search_latitude_longitude(city))
                    print("Coordinates:",(search_latitude_longitude(city)),"\n\n")
                    result.append(tup)

                else:
                    pass
        if flag==1:
            break

            
   # print(X_array)
    
    df = pd.DataFrame(result, columns=['news', 'link', 'city','lat'])  
    
    df.dropna() #removing NaN values from the dataset.
    
    #df.to_csv('static/assets/data/webscrappeddata.csv', index=False, encoding='utf-8')
    
    df1=df1.append(df)
    
    #print(df1)
    
    df1.to_csv('static/assets/data/webscrappeddata.csv', index=False, encoding='utf-8')
    
    #print("test")
    
    
    #mydata=pd.read_csv(r'webscrappeddata.csv') #file is being read.
    
    
    mydata=df1

    print(len(mydata))

    X=mydata["lat"]

    X_array=X.values.reshape(len(mydata))
    X_array=list(X_array)

    print("Data Collected:",X_array)


    data=[]

    for i in X_array:
        i=str(i)
        if (type(i) == str):
            #print("testinggggggggg")
            i=str(i[1:-1])
            val=i.split(", ")
            if(val[0] or val[1] != None ):
                data.append([float(val[0]),float(val[1])])

    #print("Final Data Extracted: /n",data)


    import folium
    from folium import plugins

    heatmap_map = folium.Map(location=[20.5937, 78.9629], zoom_start=4.5) #Map set to India using co-orodinates

    hm = plugins.HeatMap(data)
    heatmap_map.add_child(hm)

    heatmap_map.save("static/assets/heatmap_final.html")

    print("*******Done*******")
    


 --------- Page: 0 : https://www.indiatoday.in/crime?page=0 

you already have latest data
                                                  news  \
0    'Take her home or she'd be killed': Ludhiana w...   
1    Kerala: Two held for raping mentally-ill woman...   
2    Robbers pose as electricians, loot Delhi prope...   
3    Woman gang-raped at Kolkata flat, robbed of Rs...   
4    Man who stoned father to death, now tries to k...   
..                                                 ...   
174  Man kills lover's sister-in-law in Ghaziabad a...   
175  Caught on cam: Four armed men hold family host...   
176  Nagaland man borrows pistol to woo girlfriend,...   
177  Kerala girl raped as minor, accused arrested a...   
178  Kerala: Youth accused of raping minor in Idukk...   

                                                  link                 city  \
0    https://www.indiatoday.in/crime/story/-take-he...            New Delhi   
1    https://www.indiatoday.in/crime/story/kerala-k..

In [7]:
df

,news,link,city,lat
0,Conman posing as sadhu dupes jeweller's wife o...,https://www.indiatoday.in/crime/story/conman-p...,Rishikesh,"(30.1086537, 78.2916193)"
1,Andhra man who longed for a son kills 2-year-o...,https://www.indiatoday.in/crime/story/andhra-p...,New Delhi,"(28.6138954, 77.2090057)"
2,Teen booked for assaulting 4-year-old boy in U...,https://www.indiatoday.in/crime/story/teen-boo...,New Delhi,"(28.6138954, 77.2090057)"
3,BJP MLA threatens to kill SP candidate's husba...,https://www.indiatoday.in/crime/story/bjp-mla-...,Lucknow,"(26.8381, 80.9346001)"
4,Death sentence for Maharashtra man who killed ...,https://www.indiatoday.in/law/story/death-sent...,Mumbai,"(19.0759899, 72.8773928)"
5,"Minor gang-raped in UP’s Shahjahanpur, video s...",https://www.indiatoday.in/crime/story/minor-ga...,Mumbai,"(19.0759899, 72.8773928)"
6,Mumbai bus conductor gets 1 year in jail for a...,https://www.indiatoday.in/cities/mumbai/story/...,Mumbai,"(19.0759899, 72.8773928)"
7,Telangana: Woman hangs herself after killing t...,https://www.indiatoday.in/crime/story/telangan...,New Delhi,"(28.6138954, 77.2090057)"
8,Man kills lover's sister-in-law in Ghaziabad a...,https://www.indiatoday.in/crime/story/man-kill...,Ghaziabad,"(28.711241, 77.4445372)"
9,Caught on cam: Four armed men hold family host...,https://www.indiatoday.in/crime/video/caught-o...,Ghaziabad,"(28.711241, 77.4445372)"


In [8]:
df1

,news,link,city,lat
0,'Take her home or she'd be killed': Ludhiana w...,https://www.indiatoday.in/crime/story/-take-he...,New Delhi,"(28.6138954, 77.2090057)"
1,Kerala: Two held for raping mentally-ill woman...,https://www.indiatoday.in/crime/story/kerala-k...,New Delhi,"(28.6138954, 77.2090057)"
2,"Robbers pose as electricians, loot Delhi prope...",https://www.indiatoday.in/cities/delhi/story/r...,Delhi,"(28.6517178, 77.2219388)"
3,"Woman gang-raped at Kolkata flat, robbed of Rs...",https://www.indiatoday.in/cities/kolkata/story...,Kolkata,"(22.5726723, 88.3638815)"
4,"Man who stoned father to death, now tries to k...",https://www.indiatoday.in/crime/story/man-ston...,Chennai,"(13.0836939, 80.270186)"
...,...,...,...,...
161,"Man found dead in Madurai, his suicide note so...",https://www.indiatoday.in/crime/story/madurai-...,Chennai,"(13.0836939, 80.270186)"
162,Covid positive man booked for sexually assault...,https://www.indiatoday.in/crime/story/covid-po...,Bhubaneswar,"(20.2667774, 85.8435592)"
163,Man who walked 5 km with burn injuries dies in...,https://www.indiatoday.in/crime/story/man-walk...,Cuttack,"(20.4686, 85.8792)"
164,Kerala: Cops bust fake Covid-19 RT-PCR test ra...,https://www.indiatoday.in/crime/story/kerala-c...,Thiruvananthapuram,"(8.576970549999999, 77.05012463730725)"


In [9]:
df1.append(df)

,news,link,city,lat
0,'Take her home or she'd be killed': Ludhiana w...,https://www.indiatoday.in/crime/story/-take-he...,New Delhi,"(28.6138954, 77.2090057)"
1,Kerala: Two held for raping mentally-ill woman...,https://www.indiatoday.in/crime/story/kerala-k...,New Delhi,"(28.6138954, 77.2090057)"
2,"Robbers pose as electricians, loot Delhi prope...",https://www.indiatoday.in/cities/delhi/story/r...,Delhi,"(28.6517178, 77.2219388)"
3,"Woman gang-raped at Kolkata flat, robbed of Rs...",https://www.indiatoday.in/cities/kolkata/story...,Kolkata,"(22.5726723, 88.3638815)"
4,"Man who stoned father to death, now tries to k...",https://www.indiatoday.in/crime/story/man-ston...,Chennai,"(13.0836939, 80.270186)"
...,...,...,...,...
8,Man kills lover's sister-in-law in Ghaziabad a...,https://www.indiatoday.in/crime/story/man-kill...,Ghaziabad,"(28.711241, 77.4445372)"
9,Caught on cam: Four armed men hold family host...,https://www.indiatoday.in/crime/video/caught-o...,Ghaziabad,"(28.711241, 77.4445372)"
10,"Nagaland man borrows pistol to woo girlfriend,...",https://www.indiatoday.in/crime/story/nagaland...,Dimapur,"(25.9136459, 93.7283456)"
11,"Kerala girl raped as minor, accused arrested a...",https://www.indiatoday.in/crime/story/kerala-g...,Thiruvananthapuram,"(8.576970549999999, 77.05012463730725)"


In [10]:
df.append(df1)

,news,link,city,lat
0,Conman posing as sadhu dupes jeweller's wife o...,https://www.indiatoday.in/crime/story/conman-p...,Rishikesh,"(30.1086537, 78.2916193)"
1,Andhra man who longed for a son kills 2-year-o...,https://www.indiatoday.in/crime/story/andhra-p...,New Delhi,"(28.6138954, 77.2090057)"
2,Teen booked for assaulting 4-year-old boy in U...,https://www.indiatoday.in/crime/story/teen-boo...,New Delhi,"(28.6138954, 77.2090057)"
3,BJP MLA threatens to kill SP candidate's husba...,https://www.indiatoday.in/crime/story/bjp-mla-...,Lucknow,"(26.8381, 80.9346001)"
4,Death sentence for Maharashtra man who killed ...,https://www.indiatoday.in/law/story/death-sent...,Mumbai,"(19.0759899, 72.8773928)"
...,...,...,...,...
161,"Man found dead in Madurai, his suicide note so...",https://www.indiatoday.in/crime/story/madurai-...,Chennai,"(13.0836939, 80.270186)"
162,Covid positive man booked for sexually assault...,https://www.indiatoday.in/crime/story/covid-po...,Bhubaneswar,"(20.2667774, 85.8435592)"
163,Man who walked 5 km with burn injuries dies in...,https://www.indiatoday.in/crime/story/man-walk...,Cuttack,"(20.4686, 85.8792)"
164,Kerala: Cops bust fake Covid-19 RT-PCR test ra...,https://www.indiatoday.in/crime/story/kerala-c...,Thiruvananthapuram,"(8.576970549999999, 77.05012463730725)"
